__Домашняя работа к уроку №9__

Нужно реализовать rest api на базе flask
Выбрать себе датасет, сделать pipeline (преобразования + модель), сохранить его на диск. 
Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.

Я выбрал датасет про вероятность покупки клиентом страховки путешественника из домашнего задания №7. Загрузим данные.

In [1]:
!pip install dill

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('TravelInsurancePred.csv', sep=',')

df.head()

,id,Age,Emp_Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,0,31,Government Sector,Yes,400000,6,1,No,No,0
1,1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0
2,2,34,Private Sector/Self Employed,Yes,500000,4,1,No,No,1
3,3,28,Private Sector/Self Employed,Yes,700000,3,1,No,No,0
4,4,28,Private Sector/Self Employed,Yes,700000,8,1,Yes,No,0


In [4]:
df['TravelInsurance'].value_counts()

0    1277
1     710
Name: TravelInsurance, dtype: int64

Разделим данные на train/test и сохраним тестовую выборку на диск

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='TravelInsurance'), 
                                                    df['TravelInsurance'], test_size=0.3, random_state=12)
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

Напишем простой пайплайн для работы с признаками и обучения модели.

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion

In [7]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    

class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

Зададим списки признаков. Часть признаков исключаем, как малозначимые (по результатам анализа в домашней работе №7).

In [8]:
categorical_columns = ['ChronicDiseases', 'FrequentFlyer', 'EverTravelledAbroad']
continuous_columns = ['Age', 'AnnualIncome', 'FamilyMembers']

Теперь нам нужно под каждый признак создать трансформер и объединить их в список.

In [9]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    
    final_transformers.append((cont_col, cont_transformer))

Объединим все это в единый пайплайн

In [10]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Добавим классификатор и обучим наш пайплайн:

In [11]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('ChronicDiseases',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChronicDiseases')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChronicDiseases'))])),
                                                ('FrequentFlyer',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='FrequentFlyer')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='FrequentFlyer'))])),
                                                ('EverTravelledAbroad',
                                     

Сохраним модель (пайплайн)

In [12]:
with open("RF_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

#### Проверим работоспособность пайплайна.

In [13]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [14]:
with open('RF_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [15]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('ChronicDiseases',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChronicDiseases')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChronicDiseases'))])),
                                                ('FrequentFlyer',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='FrequentFlyer')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='FrequentFlyer'))])),
                                                ('EverTravelledAbroad',
                                     

In [16]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [17]:
preds[:10]

array([0.33466667, 0.21560498, 0.57480952, 0.        , 0.575     ,
       0.        , 0.45759524, 0.37544048, 1.        , 0.45      ])

Найдем наилучшую отсечку для классификации клиентов:

In [18]:
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.4544047619047618, F-Score=0.687, Precision=0.744, Recall=0.638


#### Внендрение модели

In [ ]:
!pip install flask

In [19]:
from flask import Flask, request, jsonify

In [20]:
# Проверим работоспособность Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Создадим сервис для обработки запросов к модели

In [21]:
# Загружаем обученные модели
with open('RF_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)

@app.route("/", methods=["GET"])
def general():
    return "You're using model that predicts travel insurance purchase"


@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}
    request_json = request.get_json()
    
    # проверяем корректность данных, поступивших в запросе
#     if (request_json["Age"] not in range(0,150)) or (isinstance(request_json["AnnualIncome"], numbers.Number) is False) or (request_json['FamilyMembers'] not in range(0,20)) or (request_json['ChronicDiseases'] not in [0,1]) or (request_json['FrequentFlyer'] not in ['Yes','No']) or (request_json['EverTravelledAbroad'] not in ['Yes','No']):
#         data["predictions"] = 'Wrong data to predict'
#         return jsonify(data) 
    
    Age = request_json['Age']
    AnnualIncome = request_json['AnnualIncome']
    FamilyMembers = request_json['FamilyMembers']
    ChronicDiseases = request_json['ChronicDiseases']
    FrequentFlyer = request_json['FrequentFlyer']
    EverTravelledAbroad = request_json['EverTravelledAbroad']
    
    preds = model.predict_proba(pd.DataFrame({"Age": [Age],
                                              "AnnualIncome": [AnnualIncome],
                                              "FamilyMembers": [FamilyMembers],
                                              "ChronicDiseases": [ChronicDiseases],
                                              "FrequentFlyer": [FrequentFlyer],
                                              "EverTravelledAbroad": [EverTravelledAbroad]
                                             }))
    data["predictions"] = preds[:, 1][0]
    data["success"] = True
    print('OK')

        # возвращаем словарь с данными как JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Apr/2022 12:52:20] "POST /predict HTTP/1.1" 200 -


OK


127.0.0.1 - - [25/Apr/2022 12:54:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 12:54:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 12:54:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 12:54:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 12:54:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 12:54:31] "POST /predict HTTP/1.1" 200 -


OK
OK
OK
OK
OK
OK


...

OK
OK
OK
OK
OK
OK


127.0.0.1 - - [25/Apr/2022 13:00:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 13:00:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 13:00:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 13:00:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 13:00:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 13:00:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 13:00:15] "POST /predict HTTP/1.1" 200 -


OK
OK
OK
OK
OK
OK
OK


127.0.0.1 - - [25/Apr/2022 13:00:16] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 13:00:16] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 13:00:16] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 13:00:16] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 13:00:16] "POST /predict HTTP/1.1" 200 -


OK
OK
OK
OK
OK
